In [36]:
#hypergeomteric_test_for_metabolites      


In [39]:
calculate_significance <- function(x, k, m, n){

#number of sig metabolites in specific pathway: x
#number of detected significantly changed metabolites: k
#number of metabolites in pathway: m
#number of total metabolites in total pathway: n

data_matrix <- matrix(c(x, k - x, m - x, n - k - m + x), 2, 2)
colnames(data_matrix) <- c("Sig","notSig")
rownames(data_matrix) <- c("inPathway","notPathway")
    
hyper_pval <- phyper(x - 1, k, n - k, m, lower.tail=FALSE)
fet_pval <- fisher.test(data_matrix)$p.value
                     
hyper_pval <- sprintf(hyper_pval, fmt = '%#.4f')
fet_pval <- sprintf(fet_pval, fmt = '%#.4f')                


return (fet_pval)
}


In [42]:
regulation_list = c("up","down")
# condition_list = c("cVSra","cVSneg","cVSpos","negVSpos")
condition_list = c("cVSneg","cVSpos")
data_dir = "../../../../analysis/statistics/gse/metabolomics/"

for (i in 1:length(condition_list)){
    
    condition = condition_list[i]
    
    for (j in 1:length(regulation_list)){
        regulation = regulation_list[j]
        
        file_name = paste(data_dir, "metabolomics.", condition, ".", regulation,".hypergeometric.ready.tsv", sep="")
        file_df = read.csv(file_name,sep="\t",header=TRUE)
        r = dim(file_df)[1]
        
        output_txt = paste(data_dir, "metabolomics.", condition, ".", regulation,".hypergeometric.result.tsv", sep="")
        
        if (file.exists(output_txt)) {#Delete file if it exists
            file.remove(output_txt)
        }
        
        output_string = 'pathway\tpvalue\tnumber_of_metabolites_in_pathway(m)\tnumber_of_hits(x)\tmetabolite_list\n'        
        cat(output_string, file=output_txt, append=TRUE)
 
        for (index in 1:r){
            pathway = file_df[index,1]
            m = file_df[index,2]
            x = file_df[index,3]
            n = file_df[index,4]
            k = file_df[index,5]
            # print (k)
            hit_metabolites = file_df[index,6]

            fet_pval = calculate_significance(x, k, m, n)       
            output_string = paste(pathway, '\t', fet_pval, 
                                  '\t', m, '\t', x, '\t', hit_metabolites, "\n", sep="")
            cat(output_string, file=output_txt, append=TRUE)
        }
        # break
    }
    # break
}
